In [ ]:
#Öğrenci Bilgilendirme Sistemi

In [ ]:
"""
Öğrenciler
  Dersler -> Notlar -> Midterm -> Final
  Sınıflara Göre Ders Ortalaması
  Bölüm,
  Devamsızlık Bilgisi,
  Giriş Yılı
"""

1. Kullanıcılar

2. Dersler

3. Notlar

## Veritabanı

In [ ]:
import sqlite3

In [ ]:
baglanti = sqlite3.connect("obs.db")

In [ ]:
class SqlQueries():
  def __init__(self, sqlQueries,operation="r"):
    self.sqlQueries = sqlQueries
    self.operation = operation

In [ ]:
class SqlQuery():
  def __init__(self,query,parameters=()):
    self.query=query
    self.parameters=parameters

In [ ]:
class DbOperations():
  def __init__(self,baglanti):
    self.baglanti=baglanti
    self.cursor = baglanti.cursor()

  def ExecuteSql(self,sql_queries):
    try:
      print(sql_queries.operation)
      for sql_query in sql_queries.sqlQueries:
        print(sql_query.query)
        self.cursor.execute(sql_query.query,sql_query.parameters)
      if sql_queries.operation == "r":
        return self.cursor.fetchall()
      else:
        self.baglanti.commit()
    except Exception as e:
      print(str(e))


  def InitializeDatabase(self):
    self.CreateDepartmentsTable()
    self.CreateUsersTable()
    self.CreateLessonsTable()
    self.CreateGradesTable()

  def InitializeRecords(self):
    self.CreateDepartmentRecords()
    self.CreateUserRecords()
    self.CreateLessonRecords()
    

  def CreateUsersTable(self):
    
    query='''
      CREATE TABLE IF NOT EXISTS Users(
      Id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
      Name VARCHAR(30) NOT NULL,
      Surname VARCHAR(30) NOT NULL,
      Username VARCHAR(60) UNIQUE NOT NULL,
      Password VARCHAR(40) NOT NULL,
      DepartmentId INTEGER NOT NULL,
      Policy BIT NOT NULL,

      FOREIGN KEY(DepartmentId) REFERENCES Departments(Id) 
      );
      '''
      
    sql_query = SqlQuery(query=query)
    sql_queries = SqlQueries([sql_query],operation="c")

    self.ExecuteSql(sql_queries) #Policy = 1 Öğrenci , Policy = 0 Öğretmen
      
  def CreateLessonsTable(self):
    query = '''
      CREATE TABLE IF NOT EXISTS Lessons(
        Id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        Name VARCHAR(50) NOT NULL UNIQUE,
        Code VARCHAR(10) NOT NULL UNIQUE,
        Midterm1Effect INTEGER NOT NULL,
        Midterm2Effect INTEGER NOT NULL,
        FinalEffect INTEGER NOT NULL,
        DepartmentId INTEGER NOT NULL,


        FOREIGN KEY(DepartmentId) REFERENCES Departments(Id) 
      );
    '''

    sql_query = SqlQuery(query=query)
    sql_queries = SqlQueries([sql_query],operation="c")
    self.ExecuteSql(sql_queries)


  def CreateDepartmentsTable(self):
    query = '''
    CREATE TABLE IF NOT EXISTS Departments(
      Id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
      Name VARCHAR(50) UNIQUE NOT NULL
    );
    '''

    sql_query = SqlQuery(query=query)
    sql_queries = SqlQueries([sql_query],operation="c")
    self.ExecuteSql(sql_queries)

  def CreateGradesTable(self):
    query = '''
      CREATE TABLE IF NOT EXISTS Grades(
        Id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,

        UserId INTEGER NOT NULL,
        LessonId INTEGER NOT NULL,

        Midterm1 INTEGER NULL,
        Midterm2 INTEGER NULL,
        Final INTEGER NULL,
        ExtraNote INTEGER NULL,

        GeneralGrade INTEGER NULL,

        FOREIGN KEY(UserId) REFERENCES Users(Id),
        FOREIGN KEY(LessonId) REFERENCES Lessons(Id)
      );
    '''
    
    sql_query = SqlQuery(query=query)
    sql_queries = SqlQueries([sql_query],operation="c")
    self.ExecuteSql(sql_queries)



  def CreateUserRecords(self):
      ce_sql_query = SqlQuery(query = "SELECT Id FROM Departments WHERE Name = ?",parameters=("Computer Engineering",))
      ie_sql_query = SqlQuery(query = "SELECT Id FROM Departments WHERE Name = ?",parameters=("Industrial Engineering",))
      me_sql_query = SqlQuery(query = "SELECT Id FROM Departments WHERE Name = ?",parameters=("Mechanical Engineering",))

      ce_id = self.ExecuteSql(SqlQueries([ce_sql_query]))[0]
      ie_id = self.ExecuteSql(SqlQueries([ie_sql_query]))[0]
      me_id = self.ExecuteSql(SqlQueries([me_sql_query]))[0]

      print(ce_id)
      print(ie_id)
      print(me_id)
      query_list=[]
      
      query_list.append(SqlQuery('''
      INSERT INTO Users(Name,Surname,Username,Password,Department,Policy)
      VALUES ('Fethi','Tekyaygil','fethitekyaygil','123',?,1)
      ''',(ce_id)))

      query_list.append(SqlQuery('''
      INSERT INTO Users(Name,Surname,Username,Password,Department,Policy)
      VALUES ('Taha','Tekyaygil','tahatekyaygil','123456',?,1)
      ''',(ce_id)))

      query_list.append(SqlQuery('''
      INSERT INTO Users(Name,Surname,Username,Password,Department,Policy)
      VALUES ('Enes','Ağdağ','enesagdag','123432',?,1)
      ''',(ie_id)))
      
      query_list.append(SqlQuery('''
      INSERT INTO Users(Name,Surname,Username,Password,Department,Policy)
      VALUES ('Murat','Özcan','muratozcan','123213',?,1)
      ''',(me_id)))
      
      query_list.append(SqlQuery('''
      INSERT INTO Users(Name,Surname,Username,Password,Department,Policy)
      VALUES ('Hıdır','Sevinç','hidirsevinç','123333',?,0)
      ''',(ce_id)))
      
      query_list.append(SqlQuery('''
      INSERT INTO Users(Name,Surname,Username,Password,Department,Policy)
      VALUES ('Yeşim','Çalık','yesimcalik','1233334',?,0)
      ''',(ie_id)))

      query_list.append(SqlQuery('''
      INSERT INTO Users(Name,Surname,Username,Password,Department,Policy)
      VALUES ('Sefer','Kurnaz','seferkurnaz','1233331212',?,0)
      ''',(me_id)))
      
      
      self.ExecuteSql(SqlQueries(query_list,operation="i")) #i = insert



  def CreateDepartmentRecords(self):
      query_list=[]

      query_list.append(SqlQuery('''
      INSERT INTO Departments(Name) VALUES('Computer Engineering');
      '''))

      query_list.append(SqlQuery('''
      INSERT INTO Departments(Name) VALUES('Industrial Engineering');
      '''))

      query_list.append(SqlQuery('''
      INSERT INTO Departments(Name) VALUES('Mechanical Engineering');
      '''))

      self.ExecuteSql(SqlQueries(query_list,operation="i"))


  def CreateLessonRecords(self):
      query_list=[]

      query_list.append(SqlQuery('''
      INSERT INTO Lessons(Name,Code,Midterm1Effect,Midterm2Effect,FinalEffect)
      VALUES ('Python Programlama 1','PY101','30','35','35')
      '''))

      query_list.append(SqlQuery('''
      INSERT INTO Lessons(Name,Code,Midterm1Effect,Midterm2Effect,FinalEffect)
      VALUES ('Python Programlama 1','PY101','30','35','35')
      '''))


      query_list.append(SqlQuery('''
      INSERT INTO Lessons(Name,Code,Midterm1Effect,Midterm2Effect,FinalEffect)
      VALUES ('Python Programlama 1','PY101','30','35','35')
      '''))


      query_list.append(SqlQuery('''
      INSERT INTO Lessons(Name,Code,Midterm1Effect,Midterm2Effect,FinalEffect)
      VALUES ('Python Programlama 1','PY101','30','35','35')
      '''))

      self.ExecuteSql(SqlQueries(query_list,operation="i"))

  def GetUsers(self):
    print(self.ExecuteSql(SqlQueries([SqlQuery("SELECT * FROM Users")])))

  def GetDepartments(self):
    print(self.ExecuteSql(SqlQueries([SqlQuery("SELECT * FROM Departments")])))

In [ ]:
db_ops = DbOperations(baglanti)

In [ ]:
db_ops.InitializeDatabase()
db_ops.InitializeRecords()

c

    CREATE TABLE IF NOT EXISTS Departments(
      Id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
      Name VARCHAR(50) UNIQUE NOT NULL
    );
    
c

      CREATE TABLE IF NOT EXISTS Users(
      Id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
      Name VARCHAR(30) NOT NULL,
      Surname VARCHAR(30) NOT NULL,
      Username VARCHAR(60) UNIQUE NOT NULL,
      Password VARCHAR(40) NOT NULL,
      DepartmentId INTEGER NOT NULL,
      Policy BIT NOT NULL,

      FOREIGN KEY(DepartmentId) REFERENCES Departments(Id) 
      );
      
c

      CREATE TABLE IF NOT EXISTS Lessons(
        Id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        Name VARCHAR(50) NOT NULL UNIQUE,
        Code VARCHAR(10) NOT NULL UNIQUE,
        Midterm1Effect INTEGER NOT NULL,
        Midterm2Effect INTEGER NOT NULL,
        FinalEffect INTEGER NOT NULL,
        DepartmentId INTEGER NOT NULL,


        FOREIGN KEY(DepartmentId) REFERENCES Departments(Id) 
      );
    
c

      CREATE TABLE IF NOT EXISTS Grade

In [ ]:
db_ops.GetUsers()

r
SELECT * FROM Users
[]


In [ ]:


db_ops.InitializeRecords()


Incorrect number of bindings supplied. The current statement uses 1, and there are 20 supplied.


TypeError: ignored

In [ ]:
db_ops.CreateDepartmentRecords()

In [ ]:
db_ops.GetDepartments()

[(1, 'Computer Engineering'), (2, 'Industrial Engineering'), (3, 'Mechanical Engineering')]


In [ ]:
cursor = baglanti.cursor()
cursor.execute("SELECT * FROM Users")

cursor.fetchall()

[]

 ## Backend ve Business

In [ ]:
OgrenciListesi=[]

In [ ]:
DersListesi=[]

In [ ]:
OgrenciDersNotuListesi=[]

In [ ]:
def DersEkle(isim,dersKodu,mid1Etki,mid2Etki,finalEtki,donem):
  if len([ders for ders in DersListesi if ders["DersKodu"]==dersKodu])==0:
    id = len(DersListesi)+1
    DersListesi.append({"Id":id,"Isim":isim,"DersKodu":dersKodu,
                        "Mid1Etki":mid1Etki,"Mid2Etki":mid2Etki,
                        "FinalEtki":finalEtki,"Donem":donem})

In [ ]:
#Öğrencilerimizi oluşturalım

def OgrenciEkle(adiSoyadi,ogrenciNo,sifre,bolum,devamsizlik,girisYili,donem):
  if len([eleman for eleman in OgrenciListesi if eleman["OgrenciNo"]==ogrenciNo])==0:
    OgrenciListesi.append({"AdiSoyadi":adiSoyadi,"OgrenciNo":ogrenciNo,
                           "Sifre":sifre,
                           "Bolum":bolum,
                            "Devamsizlik":devamsizlik,
                           "girisYili":girisYili,"Donem":donem})

In [ ]:
def DersEkleDuzenle(ogrenciNo,dersKodu,vize1=None,vize2=None,final=None):
  isFirst=False
  ogrenciNotuDictSorgu=[sozlukElemani for sozlukElemani in OgrenciDersNotuListesi if sozlukElemani["OgrenciNo"]==ogrenciNo and sozlukElemani["DersKodu"]==dersKodu]
  
  if len(ogrenciNotuDictSorgu)==0:
    ogrenciNotuDict={"OgrenciNo":ogrenciNo,"DersKodu":dersKodu}
    isFirst=True
  elif len(ogrenciNotuDictSorgu)>0:
    ogrenciNotuDict=ogrenciNotuDictSorgu[0]

  if vize1 is not None:
    ogrenciNotuDict["Vize1"]=vize1
    ogrenciNotuDict["Ortalama"] = OrtalamaHesapla(ogrenciNotuDict)
  if vize2 is not None:
    ogrenciNotuDict["Vize2"]=vize2
    ogrenciNotuDict["Ortalama"] = OrtalamaHesapla(ogrenciNotuDict)
  if final is not None:
    ogrenciNotuDict["Final"]=final
    ogrenciNotuDict["Ortalama"] = OrtalamaHesapla(ogrenciNotuDict)
    
  if isFirst:
     OgrenciDersNotuListesi.append(ogrenciNotuDict)

def OrtalamaHesapla(ogrenciNotuDictionaryParametre):
  ortalama=0
  ders=[ders for ders in DersListesi if ders["DersKodu"]==ogrenciNotuDictionaryParametre["DersKodu"]][0]
  if ders is not None:
    if "Vize1" in ogrenciNotuDictionaryParametre.keys():
      ortalama+=ders["Mid1Etki"]*ogrenciNotuDictionaryParametre["Vize1"]/100
    if "Vize2" in ogrenciNotuDictionaryParametre.keys():
      ortalama+=ders["Mid2Etki"]*ogrenciNotuDictionaryParametre["Vize2"]/100
    if "Final" in ogrenciNotuDictionaryParametre.keys():
      ortalama+=ders["FinalEtki"]*ogrenciNotuDictionaryParametre["Final"]/100

    return ortalama

In [ ]:
#Öğrenci        - ÖğrenciDersler (Fethi Matematik dersini alıyor ve aldığı matematik dersinde 1.vize notu 70) -               Dersler

In [ ]:
DersEkle("Python Programlama 1","PY101",20,30,50,1)
DersEkle("Matematik 1","MAT101",30,30,40,1)
DersEkle("Tarih 1","HIST101",25,30,45,1)


DersEkle("Python Programlama 2","PY102",30,35,35,2)
DersEkle("Veri Yapısı ve Algoritmalar 1","ALG301",25,35,40,5)

In [ ]:
OgrenciEkle("Fethi Tekyaygil","1627","123","Bilgisayar Mühendisliği",0,2014,2)
OgrenciEkle("Taha Tekyaygil","200","qwer","İşletme",0,2015,1)

In [ ]:
DersEkleDuzenle("1627","PY101",90) #Fethi Python 101 dersi alıyor
DersEkleDuzenle("1627","MAT101",75)
DersEkleDuzenle("1627","HIST101")
DersEkleDuzenle("200","PY101")

## UI

In [ ]:
def UIKullaniciyiSelamla(adSoyad):
  print(f'Hoşgeldin {adSoyad}!')

In [ ]:
def UIKullaniciIslemleri():
  secim=input("Yapmak istediğiniz işlemi seçin! \n* Dersleri görmek için 1\n* Notları görmek için 2\n ")
  if secim == "1":
    print(UIOgrenciDersleriGoruntule())
  elif secim == "2":
    UIOgrenciDersNotlariGoruntule()

In [ ]:
def HelperDersListesininSozlukOlustur():
  global ogrenciDerslerVeNotlar

  dersKodlari=[ogrenciDersNotu["DersKodu"] for ogrenciDersNotu in ogrenciDerslerVeNotlar]
  
  dersIsimleri={}

  index=1

  for dersKodu in dersKodlari:
    dersIsmi=[ders for ders in DersListesi if ders["DersKodu"]==dersKodu][0]["Isim"]
    
    dersIsimleri[str(index)]=[dersIsmi,dersKodu]

    index+=1

  return dersIsimleri

In [ ]:
def DersListesiniEkranaYazdir(dersSozluk):
  for key,value in dersSozluk.items():
    print(f"{key}. {value[0]} ({value[1]})")

In [ ]:
# Öğrenci Derslerini Görüntüle
def UIOgrenciDersleriGoruntule():
  sozluk = HelperDersListesininSozlukOlustur()
  DersListesiniEkranaYazdir(sozluk)

In [ ]:
#Başlangıçta Kullanıcının Tüm Ders ve Not Bilgisini getir
def UIBaslangictaOgrenciDersVeNotlariniGetir(ogrenciNo):
  return [ogrenciDersNotu for ogrenciDersNotu in OgrenciDersNotuListesi if ogrenciDersNotu["OgrenciNo"]==ogrenciNo]

In [ ]:
#Başlangıçta Kullanıcının Notu Olan Tüm Derslerini Filtrele
def UIBaslangictaOgrenciNotuOlanDersleriGetir():
  global ogrenciDerslerVeNotlar

  return [ders for ders in ogrenciDerslerVeNotlar if "Ortalama" in ders]

In [ ]:
#Öğrenci Ders Notlarını Görüntüle
def UIOgrenciDersNotlariGoruntule():
  secim = input("*Tüm derslerin notlarını görmek için 1\n*Tek dersin notunu görüntülemek için 2\n  ")

  if secim=="1":
    UITumDerslerinNotlariniGor()
  elif secim=="2":
    UITekDersinNotlariniGor()

In [ ]:
def HelperNotGoruntule(notListesi):
  for notuOlanDers in notListesi:
    notuOlanDersTupleList=list(notuOlanDers.items())
    for notuOlanDersTuple in notuOlanDersTupleList:
      if notuOlanDersTuple[0] != "OgrenciNo":
        print(f"{notuOlanDersTuple[0]} : {notuOlanDersTuple[1]}")
    print(" ")

In [ ]:
#Öğrencinin Tüm Derslerinin Notlarını Görüntüle
def UITumDerslerinNotlariniGor():
  HelperNotGoruntule(notuOlanDersler)

In [ ]:
def UITekDersinNotlariniGor():
  global ogrenciDerslerVeNotlar

  sozluk = HelperDersListesininSozlukOlustur()
  DersListesiniEkranaYazdir(sozluk)
  
  deger=""

  while deger not in sozluk:
    deger=input("Lütfen notunu görmek istediğini dersi seçin: ")
  
  dersKodu = sozluk[deger][1]

  secilenDers = [notuOlanDers for notuOlanDers in notuOlanDersler if notuOlanDers["DersKodu"] ==  dersKodu]

  if len(secilenDers) == 0:
    print("Bu dersin notu yok!")
  else:
    HelperNotGoruntule(secilenDers)

In [ ]:
"""
1. Tüm derslerin notlarını görüntüleyecek
2. Tek dersin notlarını görüntüleyecek
"""

# App

In [ ]:
hak=3
ogrenciNo = None
sifre = None
girisYapanOgrenci = None
ogrenciDerslerVeNotlar = []
notuOlanDersler = []

In [ ]:
while hak>0:
  ogrenciNo=input("Ogrenci no: ")
  sifre=input("Şifre: ")

  girisYapanOgrenciSorguListesi=[ogrenci for ogrenci in OgrenciListesi 
                     if ogrenci["OgrenciNo"]==ogrenciNo 
                     and ogrenci["Sifre"]==sifre]
  
  if len(girisYapanOgrenciSorguListesi)>0:
    girisYapanOgrenci=girisYapanOgrenciSorguListesi[0]

  if girisYapanOgrenci is None: 
    hak-=1
  else:
    ogrenciDerslerVeNotlar = UIBaslangictaOgrenciDersVeNotlariniGetir(ogrenciNo) #tüm dersleri getirsin
    notuOlanDersler = UIBaslangictaOgrenciNotuOlanDersleriGetir()
    UIKullaniciyiSelamla(girisYapanOgrenci["AdiSoyadi"])
    UIKullaniciIslemleri()
    break

  if hak<=0:
    print("Çok fazla hatalı deneme yaptınız lütfen öğrenci işleriyle iletişime geçin!")
    exit()